# Project 1: Training a Simple Neural Network with GPU

## Introduction

In this project, you will create, train, and evaluate a simple neural network using both TensorFlow and PyTorch. The objective is to ensure you are comfortable with setting up a neural network and utilizing GPU acceleration for training. 

The code for this assignment will mirror that of Lab 1, but you will need to complete some items, and make some changes.  You will use the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist) for this project.

## Objectives

1. Set up TensorFlow and PyTorch environments.
2. Verify GPU availability.
3. Implement a simple neural network in TensorFlow and PyTorch.
4. Train and evaluate the models.
5. Answer assessment questions.

## Instructions

Follow the steps below to complete the project. Ensure that you use a GPU to train your models.

---

### Step 1: Set Up Your Environment

First, install the necessary libraries. Run the following cell to install TensorFlow and PyTorch. 


Provide snapshots from your environment showing:
1) You are using a virtual environment
2) You have installed `TensorFlow` and `PyTorch`

---

### Step 2: Verify GPU Availability
Check if TensorFlow and PyTorch can detect the GPU.

Run the following two code blocks and show the output.

#### TensorFlow GPU Check

In [3]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

print("TensorFlow version:", tf.__version__)
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print("GPU is available for TensorFlow!")
else:
    print("No GPU found for TensorFlow.")


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.12.0
GPU is available for TensorFlow!


#### PyTorch GPU Check

In [1]:
import torch

print("PyTorch version:", torch.__version__)
if torch.cuda.is_available():
    print("GPU is available for PyTorch!")
elif torch.mps.is_available():
    print("MPS (Apple Silicon GPU support) is available for PyTorch!")
else:
    print("No GPU found for PyTorch.")


PyTorch version: 2.7.1+cu118
GPU is available for PyTorch!


---

### Step 3: Implement and Train a Simple Neural Network
#### TensorFlow Implementation
1. Load and preprocess the Fashion MNIST dataset.
2. Define the neural network model.  Departing from the example in class, use 2 hidden layers, each with 64 units and relu activation functions, densely connected.
3. Compile the model.
4. Train the model using the GPU.  Use a batch size of 64, and run 6 epochs.
5. Evaluate the model.

You need to complete and run the code. Show the complete output.


In [4]:
import psutil
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

def print_free_ram(step=""):
    ram = psutil.virtual_memory()
    print(f"[{step}] Free RAM: {ram.available / (1024 ** 3):.2f} GB")

# Track RAM after imports
# print_free_ram("After imports")

# Load and preprocess dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
# print_free_ram("After data preprocessing")

# Define model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])
# print_free_ram("After model definition")

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# print_free_ram("After model compile")

# Train model
model.fit(x_train, y_train, epochs=5, batch_size=64)
# print_free_ram("After training")

# Evaluate model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy:.4f}")
# print_free_ram("After evaluation")


Epoch 1/5
938/938 [==============================] - 3s 2ms/step - loss: 0.5535 - accuracy: 0.8114
Epoch 2/5
938/938 [==============================] - 2s 2ms/step - loss: 0.4130 - accuracy: 0.8565
Epoch 3/5
938/938 [==============================] - 2s 2ms/step - loss: 0.3745 - accuracy: 0.8674
Epoch 4/5
938/938 [==============================] - 2s 2ms/step - loss: 0.3500 - accuracy: 0.8763
Epoch 5/5
313/313 [==============================] - 1s 1ms/step - loss: 0.3717 - accuracy: 0.8682
Test accuracy: 0.8682


#### PyTorch Implementation
1. Load and preprocess the Fashion MNIST dataset.  Set batch size to 64.
2. Define the neural network model.  Departing from the example in class, use 2 hidden layers, each with 64 units and relu activation functions, densely connected.  
4. Define loss function and optimizer.
5. Train the model using the GPU.  Run 6 epochs.
6. Evaluate the model.

You need to complete and run the code. Show the complete output.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Load and preprocess the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)

# Define the model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)  # 28x28 input flattened → 128 neurons
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)  
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

model = SimpleNN()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Check for GPU
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)
# Check for GPU
if torch.cuda.is_available():
    dev = 'cuda'
elif torch.mps.is_available():
    dev = 'mps'
else:
    dev = 'cpu'
device = torch.device(dev)
model.to(device)


# Train the model
num_epochs = 6
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total    
    print(f'Test Accuracy: {accuracy:.2f}%')


Epoch [1/6], Loss: 0.4691
Epoch [2/6], Loss: 0.1514
Epoch [3/6], Loss: 0.3949
Epoch [4/6], Loss: 0.2682
Epoch [5/6], Loss: 0.2422
Epoch [6/6], Loss: 0.1963
Test Accuracy: 87.03%


---
### Questions
Answer the following questions in detail.

1. What is the purpose of normalizing the input data in both TensorFlow and PyTorch implementations?
2. Explain the role of the activation function relu in the neural network.
3. Why is it important to use GPU for training neural networks?
4. Compare the training time and accuracy of the TensorFlow and PyTorch models. Which one performed better and why?


---
### Submission
Submit a link to your completed Jupyter Notebook (e.g., on GitHub (private) or Google Colab) with all the cells executed, and answers to the assessment questions included at the end of the notebook.